## Tugas

1. Menggunakan data (WBC) dan alogirtma yang sama (DT) pada jobsheet js11_02, silahkan implemnetasikan ulang teknik *exhausive search* dengan menggunakan ***random search***. Scikit-learn sudah menyediakan fungsi ini.

2. Menggunakan data yang sama, lakukan tunning dengan menggunakan algoritma SVM. Hyperparamter yang akan ditunning adalah,
    - kernel (linier, polinomial, rbf)
    - degree (akan berdampak jika polinomial)
    - koefisien C (regularization)

In [2]:
# Import Libraries
import pandas as pd
import numpy as np


In [3]:
# Import Dataset
dataset = pd.read_csv('data/wbc.csv')

In [4]:
# show dataset
dataset.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [5]:
# remove last column because it's unnecessary
dataset = dataset.iloc[:, :-1]

In [6]:
# split label and features
X = dataset.iloc[:, 2:].values
y = dataset.iloc[:, 1].values

# shape of X and y
print(X.shape)
print(y.shape)

(569, 30)
(569,)


In [17]:
# Split dataset into training set and test set
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Split dataset into training set and validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=0)



# Create Repeated Stratiefied K-Fold
This is used so that the dataset will be tested either for testing or training.

In [18]:
# create RepeatedStratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=4, n_repeats=3, random_state=1)

In [19]:
# import library
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV

# create DecisionTreeClassifier
clf = DecisionTreeClassifier()

# param dist
param_dist = {"max_depth": [3, None],
                "max_features": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                "min_samples_split": [2, 3, 4, 5, 6, 7, 8, 9, 10],
                "min_samples_leaf": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                "criterion": ["gini", "entropy", "log_loss"]}

# implement random search
random_search = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=100, cv=cv, scoring='accuracy', n_jobs=-1, random_state=0)

# fit model with validation set
random_search.fit(X_val, y_val)

# best score
print('Best score: {}'.format(random_search.best_score_))

# best params
print('Best params: {}'.format(random_search.best_params_))



Best score: 0.938888888888889
Best params: {'min_samples_split': 7, 'min_samples_leaf': 1, 'max_features': 10, 'max_depth': None, 'criterion': 'gini'}


In [20]:
# create DecisionTreeClassifier with best params
clf = DecisionTreeClassifier(criterion='gini', max_depth=None, max_features=10, min_samples_leaf=1, min_samples_split=7)

# fit model with training set
clf.fit(X_train, y_train)

# predict with test set
y_pred = clf.predict(X_test)

# import library
from sklearn.metrics import accuracy_score

# accuracy score
print('Accuracy score: {}'.format(accuracy_score(y_test, y_pred)))



Accuracy score: 0.9122807017543859


# SVC Using GridSearchCV

In [21]:
# import dependencies SVM and GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# create SVC
svc = SVC()

# param grid
param_grid = {'C': [0.1, 1, 10, 100, 1000],
                'kernel': ["linear", "poly", "rbf" ], 
                'degree': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],}
# implement grid search
grid_search = GridSearchCV(svc, param_grid, cv=cv, scoring='accuracy', n_jobs=-1)

# fit model using validation data
grid_search.fit(X_val, y_val)

# best score
print('Best score: {}'.format(grid_search.best_score_))

# best params
print('Best params: {}'.format(grid_search.best_params_))



Best score: 0.9555555555555556
Best params: {'C': 0.1, 'degree': 1, 'kernel': 'linear'}


In [23]:
# create SVC with best params
svc = SVC(C=0.1, degree=1, kernel='linear')

# fit model with training set
svc.fit(X_train, y_train)

# predict with test set
y_pred = svc.predict(X_test)

# accuracy score
print('Accuracy score: {}'.format(accuracy_score(y_test, y_pred)))



Accuracy score: 0.9239766081871345


# SVC Using Random Search

In [24]:
# SVC Using Random Search

# create SVC
svc2 = SVC()

# param dist
param_dist = {'C': [0.1, 1, 10, 100, 1000],
                'kernel': ["linear", "poly", "rbf" ],
                'degree': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],}

# implement random search
random_search2 = RandomizedSearchCV(svc2, param_distributions=param_dist, n_iter=100, cv=cv, scoring='accuracy', n_jobs=-1, random_state=0)

# fit model with validation set
random_search2.fit(X_val, y_val)

# best score
print('Best score: {}'.format(random_search2.best_score_))

# best params
print('Best params: {}'.format(random_search2.best_params_))

Best score: 0.9555555555555556
Best params: {'kernel': 'linear', 'degree': 9, 'C': 0.1}


In [25]:
# create SVC with best params
svc2 = SVC(C=0.1, degree=9, kernel='linear')

# fit model with training set
svc2.fit(X_train, y_train)

# predict with test set
y_pred = svc2.predict(X_test)

# accuracy score
print('Accuracy score: {}'.format(accuracy_score(y_test, y_pred)))

Accuracy score: 0.9239766081871345


# 